Создание копуса с параллельными контекстами (ориганл + три перевода)

In [ ]:
import re
import numpy as np
from scipy import spatial
import pandas as pd
import os
from tqdm import tqdm
import razdel
from sentence_transformers import SentenceTransformer

In [ ]:

# Папка с файлами
base_dir = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\06.02\3" 

# Файлы
en_file = os.path.join(base_dir, "orig_a_3_1.txt")  
ru_files = [
    os.path.join(base_dir, "1_a_3_1.txt"),   # Кистяковский Муравьев
    os.path.join(base_dir, "2_a_3_1.txt"),   # Грушецкий Григорьева
    os.path.join(base_dir, "3_a_3_1.txt")    # Каррик Каменкович
]

# Куда сохранить результат
output_file = os.path.join(base_dir, "par_corp_a_3_1.xlsx") 


In [ ]:
# Очистка текста

def preprocess_text(text):
    
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Читаем английский
with open(en_file, 'r', encoding='utf-8') as f:
    en_text = preprocess_text(f.read())

# Читаем русские переводы
ru_texts = []
for ru_file in ru_files:
    with open(ru_file, 'r', encoding='utf-8') as f:
        ru_texts.append(preprocess_text(f.read()))

print(f"Английский: {len(en_text)} символов")
for i, text in enumerate(ru_texts, 1):
    print(f"Перевод {i}: {len(text)} символов")


Английский: 35443 символов
Перевод 1: 34037 символов
Перевод 2: 28924 символов
Перевод 3: 36322 символов


In [ ]:
# Разделяем на предложения

def split_english_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZА-Я])', text)
    return [s.strip() for s in sentences if s.strip()]

# Английские предложения
en_sentences = split_english_sentences(en_text)

# Русские предложения
ru_sentences_list = []
for i, ru_text in enumerate(ru_texts, 1):
    sentences = [sentence.text for sentence in razdel.sentenize(ru_text)]
    ru_sentences_list.append(sentences)
    print(f"  Перевод {i}: {len(sentences)} предложений")


print(f"Английский: {len(en_sentences)} предложений")

  Перевод 1: 365 предложений
  Перевод 2: 414 предложений
  Перевод 3: 508 предложений
Английский: 275 предложений


In [ ]:
# Создаем эмбеддинги

model = SentenceTransformer('distiluse-base-multilingual-cased')

# Эмбеддинги для английского
en_vectors = model.encode(en_sentences, batch_size=32, show_progress_bar=True)

# Эмбеддинги для русских переводов
ru_vectors_list = []
for i, ru_sentences in enumerate(ru_sentences_list, 1):
    print(f"Перевод {i}...")
    vectors = model.encode(ru_sentences, batch_size=32, show_progress_bar=True)
    ru_vectors_list.append(vectors)

Batches: 100%|██████████| 9/9 [01:28<00:00,  9.80s/it]


Перевод 1...


Batches: 100%|██████████| 12/12 [02:05<00:00, 10.43s/it]


Перевод 2...


Batches: 100%|██████████| 13/13 [01:49<00:00,  8.40s/it]


Перевод 3...


Batches: 100%|██████████| 16/16 [02:08<00:00,  8.05s/it]


In [ ]:
# Выравниваем с первым переводом (КистяМур)

alignments = []
used_ru = set()

# Берем первый перевод как базовый
ru_sentences_1 = ru_sentences_list[0]
ru_vectors_1 = ru_vectors_list[0]

for i in tqdm(range(len(en_sentences)), desc="Поиск соответствий"):
    best_j = -1
    best_score = 0
    
    # Определяем окно поиска
    k = len(en_sentences) / len(ru_sentences_1)
    j_start = max(0, int((i - 20) / k))
    j_end = min(len(ru_sentences_1), int((i + 20) / k) + 1)
    
    # Ищем лучшее соответствие
    for j in range(j_start, j_end):
        if j in used_ru:
            continue
        
        sim = 1 - spatial.distance.cosine(en_vectors[i], ru_vectors_1[j])
        if sim > best_score:
            best_score = sim
            best_j = j
    
    # Если нашли хорошее соответствие
    if best_score >= 0.65 and best_j != -1:
        alignments.append({
            'pair_id': len(alignments) + 1,
            'en_idx': i,
            'english': en_sentences[i],
            'russian_1': ru_sentences_1[best_j],
            'sim_1': best_score
        })
        used_ru.add(best_j)

print(f"Найдено {len(alignments)} пар")

Поиск соответствий: 100%|██████████| 452/452 [00:01<00:00, 235.26it/s]

Найдено 124 пар


In [ ]:
# Ищем соответствия в других переводах

# Имена переводчиков
translator_names = ["2", "3"]

for align in tqdm(alignments, desc="Обработка пар"):
    en_idx = align['en_idx']
    
    # Для каждого перевода (кроме первого)
    for trans_idx in range(1, 3):
        ru_sentences = ru_sentences_list[trans_idx]
        ru_vectors = ru_vectors_list[trans_idx]
        trans_name = translator_names[trans_idx-1]
        
        best_j = -1
        best_score = 0
        
        # Определяем окно поиска
        k = len(en_sentences) / len(ru_sentences)
        j_start = max(0, int((en_idx - 20) / k))
        j_end = min(len(ru_sentences), int((en_idx + 20) / k) + 1)
        
        # Ищем лучшее соответствие
        for j in range(j_start, j_end):
            sim = 1 - spatial.distance.cosine(en_vectors[en_idx], ru_vectors[j])
            if sim > best_score:
                best_score = sim
                best_j = j
        
        # Добавляем в результат
        align[f'russian_{trans_name}'] = ru_sentences[best_j] if best_j != -1 else ''
        align[f'sim_{trans_name}'] = best_score if best_j != -1 else 0



Обработка пар: 100%|██████████| 124/124 [00:01<00:00, 121.48it/s]


In [ ]:
# Создаем DataFrame
data = []
for align in alignments:
    row = {
        'pair_id': align['pair_id'],
        'english': align['english'],
        'russian_1': align['russian_1'],
        'russian_2': align.get('russian_2', ''),
        'russian_3': align.get('russian_3', ''),
        'sim_1': align.get('sim_1', 0),
        'sim_2': align.get('sim_2', 0),
        'sim_3': align.get('sim_3', 0)
    }
    data.append(row)

df = pd.DataFrame(data)

# Сохраняем в Excel
df.to_excel(output_file, index=False)


In [ ]:
print(f"Создано параллельных контекстов: {len(df)}")
print(f"Сохранено в: {output_file}")

print("\nПервые 3 строки:")
print(df[['pair_id', 'english', 'russian_1']].head(3))

print("\nКолонки в файле:")
print(df.columns.tolist())



Создано параллельных контекстов: 124
Сохранено в: C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\06.02\2\par_corp_a_2_1.xlsx

Первые 3 строки:
   pair_id                                            english  \
0        1  Two or three weeks had passed, and still Frodo...   
1        2                         I am getting very anxious.   
2        3  But you are leaving the Shire — and that shoul...   

                                           russian_1  
0  С первого разговора прошло уже две или три нед...  
1                         Что-то мне очень тревожно.  
2  Но Хоббитанию ты покидаешь – и надо, чтобы об ...  

Колонки в файле:
['pair_id', 'english', 'russian_1', 'russian_2', 'russian_3', 'sim_1', 'sim_2', 'sim_3']


Делаем то же самое с другими главами